In [ ]:
from google.colab import files

uploaded = files.upload()

Saving area_0_train.csv to area_0_train.csv
Saving area_0_valid.csv to area_0_valid.csv
Saving area_1_train.csv to area_1_train.csv
Saving area_1_valid.csv to area_1_valid.csv
Saving area_2_train.csv to area_2_train.csv
Saving area_2_valid.csv to area_2_valid.csv
Saving area_3_train.csv to area_3_train.csv
Saving area_3_valid.csv to area_3_valid.csv
Saving area_4_train.csv to area_4_train.csv
Saving area_4_valid.csv to area_4_valid.csv
Saving area_5_train.csv to area_5_train.csv
Saving area_5_valid.csv to area_5_valid.csv
Saving area_6_train.csv to area_6_train.csv
Saving area_6_valid.csv to area_6_valid.csv
Saving area_7_train.csv to area_7_train.csv
Saving area_7_valid.csv to area_7_valid.csv


In [ ]:
pip install torchviz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4147 sha256=38727c1ff4792992c61d0dc558be4280bfff6949348b4e457f02b689cf20358e
  Stored in directory: /root/.cache/pip/wheels/29/65/6e/db2515eb1dc760fecd36b40d54df65c1e18534013f1c037e2e
Successfully built torchviz


In [ ]:
pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# main v4
F1 Score:  0.06666666666666667
Accuracy:  0.7741935483870968
Cnfusion Matrix:
[[190  32]
 [ 24   2]]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#    from models import NoisyRNN

import glob
import random

seed = 123
random.seed(123)
torch.manual_seed(seed)

files =  glob.glob('area*.csv')
train_files = [f for f in files if 'train' in f]
test_files = [f for f in files if 'val' in f]
#print(len(train_files), len(test_files))

##
# Define Model
##
k = 2
num_layers = 1
class Model(nn.Module):

    def __init__(self):
        super(Model,self).__init__()
        self.rnn = nn.RNN(48,k,num_layers)
        self.linear = nn.Linear(k,2)

    def forward(self,x):
        h0 = torch.randn(num_layers, x.size()[1], k)
        output, hn = self.rnn(x, h0)
        output = output[:,-1,:]
        #print(output.size())
        #exit()
        #outputs = torch.stack(outputs,dim=0)
        outputs = F.leaky_relu(self.linear(output))
        outputs = outputs.view(-1,2)
        #outputs = torch.argmax(outputs,dim=0)
        return outputs







##
# Read in Data
##
x0_train = []
x1_train = []
y = []
for train_file in train_files:
    with open(train_file, 'r') as f:
        f.readline()
        data = []
        targets = []
        for i,line in enumerate(f):
            line = line.strip('\n').split(',')
            print(i)
            #print(line)
            dat = line[2:-3]
            dat = [float(d) for d in dat]
            mag = float(line[-3])
            #print(mag)
            if mag == 0.0:
                target = 0
            else:
                target = 1
            long_ = float(line[-2])
            lat_ = float(line[-1])
            data.append(dat)
            targets.append(target)
        for i in range(len(data) - k - 1):
            if targets[i+k+1] == 0:
                x0_train.append((data[i:i+k], targets[i+k+1]))
            else:
                x1_train.append((data[i:i+k], targets[i+k+1]))

import random
x0_train = random.sample(x0_train, len(x1_train))
x_train = x0_train + x1_train
print('x_train length',len(x_train), x_train)

random.shuffle(x_train)
X_train = [i[0] for i in x_train]
y = [i[1] for i in x_train]

##
# Read in Val Data (Test)
##
x0_test = []
x1_test = []
for test_file in test_files:
    with open(test_file, 'r') as f:
        f.readline()
        data = []
        targets = []
        for i,line in enumerate(f):
            line = line.strip('\n').split(',')
            #print(line)
            dat = line[2:-3]
            dat = [float(d) for d in dat]
            mag = float(line[-3])
            if mag == 0.0:
                target = 0
            else:
                target = 1
            long_ = float(line[-2])
            lat_ = float(line[-1])
            data.append(dat)
            targets.append(target)
        for i in range(len(data) - k - 1):
            if targets[i+k+1] == 0:
                x0_test.append((data[i:i+k], targets[i+k+1]))
            else:
                x1_test.append((data[i:i+k], targets[i+k+1]))

#x0_test = random.sample(x0_test, len(x1_test))
x_test = x0_test + x1_test
X_test = [i[0] for i in x_test]
y_test = [i[1] for i in x_test]

###
# Run Model on Train Data
###
model = Model()
max_epochs = 10
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(max_epochs):
    x_ = torch.tensor(X_train).float()
    #print(X_train.size())
    y_ = torch.tensor(y).long()
    optimizer.zero_grad()
    output = model(x_)
    y_ = y_.view(-1)
    #print(output.size(),y.size())
    loss = loss_fn(output,y_)
    loss.backward()
    optimizer.step()
    #print('Done')

###
# Run Model on Val Data
###
model.eval()
X_test = torch.tensor(X_test).float()
output = model(X_test)
pred = list(torch.argmax(output,dim=1).numpy())

###
# Evaluation
###
assert len(pred) == len(y_test) , "pred and true must be same size"
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

score_f1 = f1_score(y_test, pred)
score_acc = accuracy_score(y_test, pred)
c_matrix = confusion_matrix(y_test, pred)

print("F1 Score: ", score_f1)
print("Accuracy: ", score_acc)
print("Cnfusion Matrix: ")
print(c_matrix)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136


In [ ]:
26+34+188

248

# min v5
Using the CPU
F1 Score:  0.0
Accuracy:  0.5
Cnfusion Matrix:
[[26  0]
 [26  0]]

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from torch._utils import _accumulate
from torch.utils.data import Subset


def get_device():
    """Get a gpu if available."""
    if torch.cuda.device_count()>0:
        device = torch.device('cuda')
        print("Connected to a GPU")
    else:
        print("Using the CPU")
        device = torch.device('cpu')
    return device

def which_device(model):
    return next(model.parameters()).device


def add_channels(X):
    if len(X.shape) == 2:
        return X.reshape(X.shape[0], 1, X.shape[1],1)

    elif len(X.shape) == 3:
        return X.reshape(X.shape[0], 1, X.shape[1], X.shape[2])

    else:
        return "dimenional error"




def exp_lr_scheduler(epoch, optimizer, strategy='normal', decay_eff=0.1, decayEpoch=[]):
    """Decay learning rate by a factor of lr_decay every lr_decay_epoch epochs"""

    if strategy=='normal':
        if epoch in decayEpoch:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= decay_eff
            print('New learning rate is: ', param_group['lr'])
    else:
        print('wrong strategy')
        raise ValueError('A very specific bad thing happened.')

    return optimizer



def gaussian_init_(n_units, std=1):
    sampler = torch.distributions.Normal(torch.Tensor([0]), torch.Tensor([std/n_units]))
    A_init = sampler.sample((n_units, n_units))[..., 0]
    return A_init


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from copy import deepcopy

#from tools import *


class NoisyRNN(nn.Module):
    def __init__(self, input_dim, output_classes, n_units=128, eps=0.01,
                 beta=0.8, gamma_A=0.01, gamma_W=0.01, init_std=1, alpha=1,
                 solver='base', add_noise=0, mult_noise=0):
        super(NoisyRNN, self).__init__()

        self.device = get_device()


        self.n_units = n_units
        self.eps = eps
        self.solver = solver
        self.beta = beta
        self.alpha = alpha
        self.gamma_A = gamma_A
        self.gamma_W = gamma_W
        self.add_noise = add_noise
        self.mult_noise = mult_noise

        self.tanh = nn.Tanh()

        self.E = nn.Linear(input_dim, n_units)
        self.D = nn.Linear(n_units, output_classes)

        self.C = nn.Parameter(gaussian_init_(n_units, std=init_std))
        self.B = nn.Parameter(gaussian_init_(n_units, std=init_std))
        self.I = torch.eye(n_units).to(self.device)

        self.d = nn.Parameter(torch.rand(self.n_units).float().to(self.device)*0 + eps)


    def forward(self, x, mode='test'):
        T = x.shape[1]
        h = torch.zeros(x.shape[0], self.n_units).to(which_device(self))

        for i in range(T):
            z = self.E(x[:,i,:])

            if i == 0:
                    A = self.beta * (self.B - self.B.transpose(1, 0)) + (1-self.beta) * (self.B + self.B.transpose(1, 0)) - self.gamma_A * self.I
                    W = self.beta * (self.C - self.C.transpose(1, 0)) + (1-self.beta) * (self.C + self.C.transpose(1, 0)) - self.gamma_W * self.I


            add_noise = 0.0
            mult_noise = 1.0
            if mode == 'train':
                if self.add_noise > 0:
                    add_noise = self.add_noise * torch.randn(h.shape[0], h.shape[1]).float().to(self.device)

                if self.mult_noise > 0:
                    #mult_noise = self.mult_noise * torch.randn(h.shape[0], h.shape[1]).float().to(self.device) + 1
                    mult_noise = self.mult_noise * torch.rand(h.shape[0], h.shape[1]).float().to(self.device) + (1-self.mult_noise)


            if self.solver == 'base':
                h_update = self.alpha * torch.matmul(h, A) + self.tanh(torch.matmul(h, W) + z)
                h = h + self.eps * h_update
            elif self.solver == 'noisy':
                h_update = self.alpha * torch.matmul(h, A) + self.tanh(torch.matmul(h, W) + z)
                h = h + self.d * mult_noise * h_update + add_noise


        # Decoder
        #----------
        out = self.D(h)
        return out


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from models import NoisyRNN

import glob
import random

seed = 123
random.seed(123)
torch.manual_seed(seed)

files = glob.glob('area*.csv')
train_files = [f for f in files if 'train' in f]
test_files = [f for f in files if 'val' in f]
#print(len(train_files), len(test_files))

##
# Define Model
##
k = 2
num_layers = 1
class Model(nn.Module):

    def __init__(self):
        super(Model,self).__init__()
        # self.rnn = nn.RNN(48,k,num_layers)
        self.rnn = NoisyRNN(48,50)
        self.linear = nn.Linear(50,2)

    def forward(self,x,mode):
        h0 = torch.randn(num_layers, x.size()[1], k)
        output  = self.rnn(x, mode)
        #print(output.size())
        #exit()
        #outputs = torch.stack(outputs,dim=0)
        outputs = F.leaky_relu(self.linear(output))
        outputs = outputs.view(-1,2)
        #outputs = torch.argmax(outputs,dim=0)
        return outputs


##
# Read in Data
##
x0_train = []
x1_train = []
y = []
for train_file in train_files:
    with open(train_file, 'r') as f:
        f.readline()
        data = []
        targets = []
        for i,line in enumerate(f):
            line = line.strip('\n').split(',')
            #print(line)
            dat = line[2:-3]
            dat = [float(d) for d in dat]
            mag = float(line[-3])
            #print(mag)
            if mag == 0.0:
                target = 0
            else:
                target = 1
            long_ = float(line[-2])
            lat_ = float(line[-1])
            data.append(dat)
            targets.append(target)
        for i in range(len(data) - k - 1):
            if targets[i+k+1] == 0:
                x0_train.append((data[i:i+k], targets[i+k+1]))
            else:
                x1_train.append((data[i:i+k], targets[i+k+1]))

import random
x0_train = random.sample(x0_train, len(x1_train))
x_train = x0_train + x1_train
random.shuffle(x_train)
X_train = [i[0] for i in x_train]
y = [i[1] for i in x_train]

##
# Read in Val Data (Test)
##
x0_test = []
x1_test = []
for test_file in test_files:
    with open(test_file, 'r') as f:
        f.readline()
        data = []
        targets = []
        for i,line in enumerate(f):
            line = line.strip('\n').split(',')
            #print(line)
            dat = line[2:-3]
            dat = [float(d) for d in dat]
            mag = float(line[-3])
            if mag == 0.0:
                target = 0
            else:
                target = 1
            long_ = float(line[-2])
            lat_ = float(line[-1])
            data.append(dat)
            targets.append(target)
        for i in range(len(data) - k - 1):
            if targets[i+k+1] == 0:
                x0_test.append((data[i:i+k], targets[i+k+1]))
            else:
                x1_test.append((data[i:i+k], targets[i+k+1]))

#x0_test = random.sample(x0_test, len(x1_test))
x_test = x0_test + x1_test
X_test = [i[0] for i in x_test]
y_test = [i[1] for i in x_test]

###
# Run Model on Train Data
###
model = Model()
max_epochs = 10
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(max_epochs):
    x_ = torch.tensor(X_train).float()
    #print(X_train.size())
    y_ = torch.tensor(y).long()
    optimizer.zero_grad()
    output = model(x_,mode='train')
    y_ = y_.view(-1)
    #print(output.size(),y.size())
    loss = loss_fn(output,y_)
    loss.backward()
    optimizer.step()
    #print('Done')

###
# Run Model on Val Data
###
model.eval()
X_test = torch.tensor(X_test).float()
output = model(X_test,mode='test')
pred = list(torch.argmax(output,dim=1).numpy())

###
# Evaluation
###
assert len(pred) == len(y_test) , "pred and true must be same size"
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

score_f1 = f1_score(y_test, pred)
score_acc = accuracy_score(y_test, pred)
c_matrix = confusion_matrix(y_test, pred)

print("F1 Score: ", score_f1)
print("Accuracy: ", score_acc)
print("Cnfusion Matrix: ")
print(c_matrix)


Using the CPU
F1 Score:  0.0
Accuracy:  0.8951612903225806
Cnfusion Matrix: 
[[222   0]
 [ 26   0]]


26 correctly classified as no earthquake
26 misclassified earthquake as no earthquake

In [ ]:
248

In [ ]:
car = pd.read_csv(io.BytesIO(uploaded['car-assignments.csv']), encoding="ISO-8859–1" )
g = pd.read_csv(io.BytesIO(uploaded['gps.csv']), encoding="ISO-8859–1" )

In [ ]:
car.head()

In [ ]:
len(car)

In [ ]:
g.head()

In [ ]:
print(len(g))
mask_g = g.isnull().any(axis=1)
# filter dataframe using mask and print null rows
print(g[mask_g])

In [ ]:
merged_df = pd.merge(car, g, left_on='CarID', right_on='id')

In [ ]:
merged_df

In [ ]:
mask = merged_df.isnull().any(axis=1)
print(merged_df[mask])

In [ ]:
building_coordinate[0][range][]

[[24.88388038, 36.06553341], [24.88788038, 36.06253341]]

In [ ]:
df["location"] = ""
for i in range(len(df)):
  if (building_coordinate[0][range][0][0] < df[i].long < building_coordinate[0][range][1][0] & building_coordinate[0][range][0][1] < df[i].lat < building_coordinate[0][range][1][1]){
       df.at[i, "location"] = building_coordinate.name
  }

In [ ]:
merged_df＿check = pd.merge(car, g, left_on='CarID', right_on='id' , how='outer', indicator=True)

In [ ]:
print(merged_df＿check)

In [ ]:
import re
def parse_length(s):
    # parse field "length" into "length" ,"unit of measure"
    if pd.isnull(s):
        return (np.nan, np.nan)
    return (float(str(s)[:-1]), str(s)[-1:])

def parse_period(s):
    # parse field "period" into "period_name" ,"period start", "period end"
    try:
        parsed_groups = re.match(r'([\D]+)([\d]+)(-?)([\d]*)([\D]+)', s).groups()
    except:
        return [s, np.nan, np.nan]
    period = parsed_groups[0].strip()
    period_start = parsed_groups[1]
    period_end = parsed_groups[3] if parsed_groups[3] != '' else parsed_groups[1]
    return [period, str(period_start), str(period_end)]

def parse_named(s):
    # parse field "named_by" into "named by" ,"year of naming"
    try:
        parsed_groups = re.match(r'([^\d\(]+)(\(?)(\d+)', s).groups()
    except:
        return [s, np.nan]
    named = parsed_groups[0].strip()
    year = parsed_groups[2]
    return [named, str(year)]

In [ ]:
print(data2.shape)
data2.set_index('name', inplace=True)
data2.sample(3)

In [ ]:
print(f'{data2.type.nunique()}')
print(f'{data2.type.isna().sum()}')
data2.type.value_counts()

In [ ]:
import numpy as np # linear algebra
print(data2.type.nunique())
data2.type.value_counts()[:10]
data2.loc[data2.type=='1.0m', 'length'] = '1.0m'
data2.loc[data2.type=='1.0m','type'] = np.nan
data2.loc[data2.type.isna()]

In [ ]:
# parse column "period"
data2['period_name'], data2['period_start'], data2['period_end']  = zip(*data2['period'].apply(parse_period))
data2[['period', 'period_name', 'period_start', 'period_end']].sample(3)

In [ ]:
data2['named'], data2['named_year'] = zip(*data2['named_by'].apply(parse_named))
data2[['named_by','named', 'named_year']].sample(3)

In [ ]:
data2.loc[data2['named_year'].isnull()][['named_by','named', 'named_year','species']]

In [ ]:
data2.loc[data2['species'].isnull()].named.count()
# moved values from named_by to species
data2['species'] = np.where(data2['species'].isnull(), data2['named'], data2['species'])
data2.loc[data2['named_year'].isnull(), 'named_by'] = np.nan
data2.loc[data2['named_year'].isnull(), 'named'] = np.nan
data2.loc[data2['named_year'].isnull()][['named_by','named', 'named_year','species']]
data2['length_parsed'],data2['measure'] = zip(*data2['length'].apply(parse_length))
data2[['length','length_parsed', 'measure']].sample(3)
print(data2.measure.nunique())
data2.rename(columns={'length_parsed':'length_meters'}, inplace=True)
data2.drop(columns=['measure'], inplace=True)
data2.sample(3)

In [ ]:



# drop useless columns
df_clean = data2[['diet', 'lived_in', 'type', 'taxonomy',
       'species', 'period_name', 'period_start',
       'period_end', 'named', 'named_year',
       'length_meters']].copy()
print(df_clean.shape)
df_clean.sample(3)

In [ ]:
df_clean[df_clean['period_name'] == 'USA']

In [ ]:
import io
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['jurassic_park.csv']))
# global_development = pd.read_csv(io.BytesIO(uploaded['global_development.csv']))

In [ ]:
data

In [ ]:
# 篩選出在Early Jurassic時期的資料
df_jurassic = data[data["period_name"] == "Early Jurassic"]

# 再篩選出居住在美國的動物
df_usa_jurassic = df_jurassic[df_jurassic["lived_in"] == "USA"]

In [ ]:
df_usa_jurassic

In [ ]:
df_periods_countries = data.groupby(["period_name"]).size()

In [ ]:
print(data['diet'].value_counts())

In [ ]:
min(data['length_meters'])

In [ ]:
data[data['diet'] == 'unknown']